# The Plan-Do Loop

In [1]:
%%capture cap

# install necessary dependencies
! uv pip install duckduckgo-search
# update requirements
! pip freeze > ../requirements.txt

In [2]:
import ast
from typing import Annotated, TypedDict

from langchain_community.tools import DuckDuckGoSearchRun
from langchain_core.tools import tool
from langchain_openai import AzureChatOpenAI
from langchain_core.messages import HumanMessage

from langgraph.graph import START, StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition

@tool
def calculator(query: str) -> str:
    """A simple calculator tool. Input should be a mathematical expression."""
    return ast.literal_eval(query)

search = DuckDuckGoSearchRun()
tools = [search, calculator]
model = AzureChatOpenAI(azure_deployment="gpt-4o", api_version="2024-10-21", temperature=0.1, model="gpt-4o").bind_tools(tools)

class State(TypedDict):
    messages: Annotated[list, add_messages]

def model_node(state: State) -> State:
    res = model.invoke(state["messages"])
    return {"messages": res}

builder = StateGraph(State)
builder.add_node("model", model_node)
builder.add_node("tools", ToolNode(tools))
builder.add_edge(START, "model")
builder.add_conditional_edges("model", tools_condition)
builder.add_edge("tools","model")

graph = builder.compile()

In [3]:
graph.get_graph().draw_mermaid_png(output_file_path="../img/graph_react_agent.png")

b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x00\xd8\x00\x00\x00\xf9\x08\x02\x00\x00\x00\xca\xb2l)\x00\x00\x10\x00IDATx\x9c\xec\xdd\x07xTU\xda\x07\xf03\xbdf\x92\x99I\xef=\x81\xd0%\xb0\x86* \x88\x82\x94\xa5\x08\x8a\xa0\x9f\x80\xa0R,\xac\xa8(\x02"\xec\x82\xb0"ET\x16A\xa4\x86^d\x11$(\x10!@\x804\xd2Cz2i\xd3{\xbe7\x8c\x9b\xcd\xc6$\x12\xcc\x9d\x9c;s~O\x9ey\xee\xdc;\x13B\xe6\x9fS\xef=\x97]__\x8f\x08\xa2\xb3\xb1\x11A`\x80\x04\x91\xc0\x02\t"\x81\x05\x12D\x02\x0b$\x88\x04\x16H\x10\t,\x90 6g\xd4[\x14\xc5F\xad\xca\xa2U\x99-\xe6z\x93\x91\x06\xc3[<\x01\x93\xcde\x08]\xd8B\x17\xa6W\xa0\x00\xd1\x10\x83\x8c#\xdah\xd5\xe6\xac\x1b\xea\xdc\x14Mu\x99\xc1\xcd\x93+ta\xc1\xe7*\x91\xb1M\x06\x1a\xfc~8|fM\x19\xfc\xf1\x98!\x8e\x05\xe9\xda\xd0n\xe2\xd0\x1e\xa2\xb0\x1ebD\x1f$\x88\x08~\x03WNT\x95\xe5\xeb<\x02\xf8\xa1\xddD\xfe\x11BDgF\xbd57E]xOW\x9c\xad\x8b\x1b+\x8f\xec\xe3\x82\xe8\xc0\xd9\x83\x98\xfe\xab\xf2\xfc\xbe\n\xf8\xc0\xfa\x0c\x93"\xc7\xa2\xaa1\xc1\x1f\x18\x14\x93#gx\x8b$\xb8\xb7\xc1\x9c:\x88\x97\x0eW\xb28h\x

![ReAct Agent](../img/graph_react_agent.png)

When building agentic systems using **LangGraph**, you enable an LLM to make decisions about **which actions to take** and **when to stop**, using tools and a flexible loop-based architecture.

---

## 🧰 Tools in the Graph

In this example, we used two tools:
- `search`: to retrieve information.
- `calculator`: to perform arithmetic.

However, you can easily:
- Add more tools (e.g., database access, email, summarizers).
- Replace tools with **custom ones**.

### 🛠️ Custom Tool Example
LangGraph allows you to define and plug in **custom tools**, which can integrate external APIs, internal services, or any logic.

---

## 🧩 LangGraph Components

### 1. `ToolNode`
A **node** in the graph that:
- Executes tool calls listed in the latest **AI message**.
- Returns a `ToolMessage` with the results.
- Automatically catches exceptions and returns a **ToolMessage with the error**, allowing the LLM to respond intelligently.

### 2. `tools_condition`
A **conditional edge function** that:
- Inspects the latest **AI message**.
- Routes to the `ToolNode` **if tools are needed**.
- Ends the graph otherwise.

---

## 🔁 Looping Logic: The LLM Controls the Flow

This architecture **loops between the LLM and tools**, meaning:
- The **LLM decides** when to stop (by emitting an `output` tool call or equivalent).
- The **loop continues** until a stop condition is met.

### ✅ Why Conditional Edges?
When building loops in LangGraph:
- You’ll use **conditional edges** to define **exit logic**.
- This makes the flow **dynamic**, driven by **AI decisions** and **tool outcomes**.

---

## 📌 Summary

| Component         | Role                                                                 |
|------------------|----------------------------------------------------------------------|
| `ToolNode`       | Executes tools, returns structured results, handles errors           |
| `tools_condition`| Decides if tools should run or loop should exit                      |
| Loop             | Alternates between LLM reasoning and tool use until LLM chooses to stop |

This is the foundation of a **ReAct-style agent** in LangGraph: an intelligent, tool-using, self-controlling system. Let's see how it does with the example.

In [4]:
input = {
    "messages": [
        HumanMessage("""How old was the 30th president of the United States 
            when he died?""")
    ]
}
for c in graph.stream(input):
    print(c)

{'model': {'messages': AIMessage(content='The 30th president of the United States was Calvin Coolidge. He was born on July 4, 1872, and died on January 5, 1933. To calculate his age at the time of his death:\n\n1933 - 1872 = 61 years old.\n\nCalvin Coolidge was 61 years old when he died.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 75, 'prompt_tokens': 116, 'total_tokens': 191, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-11-20', 'system_fingerprint': 'fp_ee1d74bde0', 'id': 'chatcmpl-BaBiu1MQQMmJB6eKeRUcFaXldX7eE', 'service_tier': None, 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': '

## 🔄 Walking Through the Agent Execution

Let’s break down what happened step-by-step in the agent's run:

### 1. 🧠 Model Node Executes
- The **model node** ran first.
- It decided to invoke the `duckduckgo_search` tool based on the user's query.
- This decision triggered the **conditional edge**, which routed execution to the `tools` node.

---

### 2. 🔧 ToolNode Executes
- The `ToolNode` ran the **search tool**.
- It returned search results that included the direct answer:  
  **“Age and Year of Death: January 5, 1933 (aged 60)”**.

---

### 3. 🧠 Model Node Runs Again
- The **model was called again**, now with the search results passed in as the latest message.
- Based on this new context, it generated the **final answer** without suggesting any further tool calls.

---

### 4. ⛔ Conditional Edge Ends the Graph
- Since there were **no further tool requests**, the **conditional edge logic terminated** the graph.
- The agent recognized that it had gathered enough information and exited the loop.

---

## 🛠️ What’s Next?

Up next, we’ll explore **useful extensions** to this basic agent architecture, including ways to:
- Customize **planning logic**.
- Enhance **tool calling** capabilities.

These improvements allow for more sophisticated and adaptive agent behaviors. Let's go to the [README file](../README.md)
